### 카카오 로컬 API 활용

In [1]:
import requests
from urllib.parse import quote
import pandas as pd

- API Key 가져오기

In [2]:
with open('data/kakaoapikey.txt') as f:
    kakao_key = f.read()

- URL 만들기

In [3]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '경기도 수원시 영통구 도청로 30 (이의동)'
url = f'{base_url}?query={quote(addr)}'

In [5]:
# Header: "Authorization: KakaoAK ${REST_API_KEY}"
header = {'Authorization': f'KakaoAK {kakao_key}'}

- Kakao Local API 호출하여 결과 가져오기

In [6]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '경기 수원시 영통구 이의동 0',
    'b_code': '4111710300',
    'h_code': '4111760000',
    'main_address_no': '0',
    'mountain_yn': 'N',
    'region_1depth_name': '경기',
    'region_2depth_name': '수원시 영통구',
    'region_3depth_h_name': '광교1동',
    'region_3depth_name': '이의동',
    'sub_address_no': '',
    'x': '127.053543271',
    'y': '37.2892851628003'},
   'address_name': '경기 수원시 영통구 도청로 30',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '경기 수원시 영통구 도청로 30',
    'building_name': '경기도청',
    'main_building_no': '30',
    'region_1depth_name': '경기',
    'region_2depth_name': '수원시 영통구',
    'region_3depth_name': '이의동',
    'road_name': '도청로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.053543271',
    'y': '37.2892851628003',
    'zone_no': '16508'},
   'x': '127.053543271',
   'y': '37.2892851628003'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [7]:
result.keys()

dict_keys(['documents', 'meta'])

In [8]:
result['documents'][0]

{'address': {'address_name': '경기 수원시 영통구 이의동 0',
  'b_code': '4111710300',
  'h_code': '4111760000',
  'main_address_no': '0',
  'mountain_yn': 'N',
  'region_1depth_name': '경기',
  'region_2depth_name': '수원시 영통구',
  'region_3depth_h_name': '광교1동',
  'region_3depth_name': '이의동',
  'sub_address_no': '',
  'x': '127.053543271',
  'y': '37.2892851628003'},
 'address_name': '경기 수원시 영통구 도청로 30',
 'address_type': 'ROAD_ADDR',
 'road_address': {'address_name': '경기 수원시 영통구 도청로 30',
  'building_name': '경기도청',
  'main_building_no': '30',
  'region_1depth_name': '경기',
  'region_2depth_name': '수원시 영통구',
  'region_3depth_name': '이의동',
  'road_name': '도청로',
  'sub_building_no': '',
  'underground_yn': 'N',
  'x': '127.053543271',
  'y': '37.2892851628003',
  'zone_no': '16508'},
 'x': '127.053543271',
 'y': '37.2892851628003'}

In [9]:
result['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [10]:
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
lat, lng

(37.2892851628003, 127.053543271)

- 수원 소재 관공서 정보 완성하기

In [11]:
df = pd.read_csv('data/수원소재관공서.csv')
df

,이름,주소
0,경기도청,경기도 수원시 영통구 도청로 30 (이의동)
1,장안구청,경기도 수원시 장안구 송원로 101 (조원동)
2,권선구청,경기도 수원시 권선구 호매실로 12 (탑동)
3,팔달구청,경기도 수원시 팔달구 창룡대로 23 (매향동)
4,영통구청,경기도 수원시 영통구 효원로 407 (매탄동)


In [12]:
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [13]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,이름,주소,위도,경도
0,경기도청,경기도 수원시 영통구 도청로 30 (이의동),37.289285,127.053543
1,장안구청,경기도 수원시 장안구 송원로 101 (조원동),37.304095,127.010401
2,권선구청,경기도 수원시 권선구 호매실로 12 (탑동),37.257634,126.971898
3,팔달구청,경기도 수원시 팔달구 창룡대로 23 (매향동),37.282651,127.020079
4,영통구청,경기도 수원시 영통구 효원로 407 (매탄동),37.259599,127.046508


In [14]:
df.to_csv('data/수원소재관공서2.csv', index=False)
pd.read_csv('data/수원소재관공서2.csv')

,이름,주소,위도,경도
0,경기도청,경기도 수원시 영통구 도청로 30 (이의동),37.289285,127.053543
1,장안구청,경기도 수원시 장안구 송원로 101 (조원동),37.304095,127.010401
2,권선구청,경기도 수원시 권선구 호매실로 12 (탑동),37.257634,126.971898
3,팔달구청,경기도 수원시 팔달구 창룡대로 23 (매향동),37.282651,127.020079
4,영통구청,경기도 수원시 영통구 효원로 407 (매탄동),37.259599,127.046508
